In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import regularizers
from keras import backend as K
from keras.optimizers import Adam
from keras.layers import Input

# The architecture
model = Sequential()

model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.0001), input_shape = (995, 1177, 3)))
model.add(BatchNormalization())
model.add(Activation("elu"))

model.add(Conv2D(50, (5,5), padding='same', kernel_regularizer=regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(Activation("elu"))

model.add(Conv2D(20, (5,5), padding='same', kernel_regularizer=regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(Activation("elu"))

model.add(Conv2D(3, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(Activation("elu"))

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['cosine_proximity'])
model.summary()

Using TensorFlow backend.
c:\users\amabl\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\amabl\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\amabl\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 995, 1177, 128)    3584      
_________________________________________________________________
batch_normalization_1 (Batch (None, 995, 1177, 128)    512       
_________________________________________________________________
activation_1 (Activation)    (None, 995, 1177, 128)    0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 995, 1177, 50)     160050    
_________________________________________________________________
batch_normalization_2 (Batch (None, 995, 1177, 50)     200       
_________________________________________________________________
activation_2 (Activation)    (None, 995, 1177, 50)     0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 995, 1177, 20)    

In [2]:
# Training data ubication
commonPath = "../"
train_data_dir = commonPath + 'dataset/train'
validation_data_dir = commonPath + 'dataset/validation'

# Training configuration
nb_train_samples = 47000
nb_validation_samples = 4700
batch_size = 124
epochs = 10

In [3]:
from keras.preprocessing.image import ImageDataGenerator

"""
def generate_data(directory, batch_size):
    # Replaces Keras' native ImageDataGenerator.
    i = 0
    list_color = os.listdir(directory + "/color")
    list_gray = os.listdir(directory + "/gray")
    while True:
        image_batch_color = []
        image_batch_gray = []
        for b in range(batch_size):
            if i == len(file_list):
                i = 0
            sample_color = list_color[i]
            sample_gray = list_gray[i]
            i += 1
            
            image_color = cv2.resize(cv2.imread(sample[0]), INPUT_SHAPE)
            image_gray = cv2.resize(cv2.imread(sample[0]), INPUT_SHAPE)
            
            image_batch_color.append((image_color.astype(float) - 128) / 128)
            image_batch_gray.append((image_gray.astype(float) - 128) / 128)

        yield (np.array(image_batch_color), np.array(image_batch_gray))
"""

def generate_data(directory, batch_size):
    # Replaces Keras' native ImageDataGenerator.
    i = 0
    list_color = os.listdir(directory + "/color")
    list_gray = os.listdir(directory + "/gray")
    while True:
        image_batch_color = []
        image_batch_gray = []
        for b in range(batch_size):
            if i == len(list_color):
                i = 0
            sample_color = list_color[i]
            sample_gray = list_gray[i]
            i += 1
            
            image_color = cv2.imread(directory + "/color/" + sample_color)
            image_gray = cv2.imread(directory + "/gray/" + sample_gray)
            #print(image_gray.shape)
            #image_color = np.expand_dims(image_color, axis=0)
            #image_gray = np.expand_dims(image_gray, axis=0)

            image_batch_color.append(image_color)
            image_batch_gray.append(image_gray)

        yield (np.array(image_batch_gray), np.array(image_batch_color))

# this is the augmentation configuration we will use for training
train_datagen = generate_data(train_data_dir, 1)

# this is the augmentation configuration we will use for validation
test_datagen = generate_data(validation_data_dir, 1)

"""
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(72,128),
    batch_size=batch_size)
    #class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(72,128),
    batch_size=batch_size)
    #class_mode='categorical')
    """
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import glob
import cv2
import os,shutil
import random
import time

model.fit_generator(train_datagen,steps_per_epoch=1, epochs=100)
model.save('my_model.h5')

Epoch 1/100


UnknownError: 2 root error(s) found.
  (0) Unknown:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node conv2d_1/convolution (defined at c:\users\amabl\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:3009) ]]
	 [[batch_normalization_3/cond/else/_45/batchnorm/mul_1-1-0-TransposeNCHWToNHWC-LayoutOptimizer/_252]]
  (1) Unknown:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node conv2d_1/convolution (defined at c:\users\amabl\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:3009) ]]
0 successful operations.
0 derived errors ignored. [Op:__inference_keras_scratch_graph_4517]

Function call stack:
keras_scratch_graph -> keras_scratch_graph
